In [1]:
# import packages
import pandas as pd, numpy as np
import json
# import files
import data as dt

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [3]:
# load data
ob_krak, ob_bit = dt.read_jsonOB(file_name = 'orderbooks_05jul21.json')


In [4]:
type(ob_krak)

dict

# Alineacion de timestamps

In [9]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:51.077Z',
 '2021-07-05T13:06:52.426Z',
 '2021-07-05T13:06:53.921Z',
 '2021-07-05T13:06:55.563Z')

In [10]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]]

[]

In [12]:
10/10000

0.001

# Origin exchange (Kraken)

In [7]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [8]:
upper_krak = mid_krak*1.001
lower_krak = mid_krak*0.999
(lower_krak, upper_krak)

(28270.051649999998, 28326.648349999996)

In [9]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [10]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (30, 2), (23, 2)]

In [11]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [12]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
5,28291.2,0.745,28308.2,0.005
6,28291.0,0.396,28310.0,0.004
7,28290.9,0.340,28311.8,0.745
8,28290.8,0.006,28314.9,0.250
9,28290.0,0.003,28315.2,5.000


# Destination exchange (Bitfinex)

In [13]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

# Añadir niveles a destination

### Bids

In [14]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [15]:
# merge levels to create new OB
bids = df_bit[['bid_size','bid']].merge(levels[['bid_size','bid']], how='outer', on='bid'
                                       ).sort_values(by='bid', ascending=False, ignore_index=True)
bids.rename(columns={'bid_size_x':'bid_size','bid_size_y':'add_size'}, inplace=True)

In [16]:
# identify each order's fee structure
bids['fee_type'] = ['taker' if bid > df_bit['ask'][0] else 'maker' if pd.notna(add_size) else np.nan
                    for bid, add_size in zip(bids['bid'].values, bids['add_size'].values)]

bids.fillna(0, inplace=True)

In [17]:
# compute total volume in each OB level
bids['new_size'] = bids['bid_size'].values+bids['add_size'].values

In [18]:
df_bit[['ask','ask_size']].iloc[0,:]

ask         28278.000000
ask_size        0.011063
Name: 0, dtype: float64

In [19]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [23]:
df_bit.shape[0],levels.shape[0], bids.shape[0]

(25, 30, 55)

# Preliminary approach using loops

In [362]:
# destination exchange
bit_ask = df_bit[['ask','ask_size']]
bit_bid = df_bit[['bid_size','bid']]

# origin exchnage
krak_bid = df_krak[['bid_size','bid']]
krak_ask = df_krak[['ask','ask_size']]

In [363]:
bid_levels = levels[['bid_size','bid']]
bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
bid_levels.rename(columns={'bid_size':'size',
                          'bid':'price',
                          'bid_flag':'type'}, inplace=True)

ask_levels = levels[['ask_size','ask']]
ask_levels['ask_flag'] = ['ask']*ask_levels.shape[0]
ask_levels.rename(columns={'ask_size':'size',
                          'ask':'price',
                          'ask_flag':'type'}, inplace=True)

levels_added = bid_levels.append(ask_levels, ignore_index=True).dropna()

levels_added['transaction_time'] = np.random.uniform(1250, size=levels_added.shape[0])
levels_added.sort_values(by='transaction_time', inplace=True, ignore_index=True)

<ipython-input-363-56ab3a99147f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bid_levels['bid_flag'] = ['bid']*bid_levels.shape[0]
C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-363-56ab3a99147f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [364]:
for i in range(levels_added.shape[0]):

    to_fill = levels_added.iloc[0,:]
    
    if to_fill['type']=='bid':
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] > bit_ask.iloc[0]['ask'] else 'maker'


        if fee_type=='taker':

            bit_ask['accum_size'] = bit_ask['ask_size'].cumsum()


            vol_to_fill=to_fill['size']

            bit_ask['remaining_vol'] = vol_to_fill - bit_ask['accum_size']
            to_drop = bit_ask[bit_ask['remaining_vol']>0]

            bit_ask.drop(to_drop.index, inplace=True)
            bit_ask.reset_index(drop=True, inplace=True)

            new_vol = -bit_ask.iloc[0,-1]  # remaing volume on surviving level

            bit_ask.iloc[0,1] = new_vol
            bit_ask.drop(['accum_size', 'remaining_vol'], axis=1, inplace=True) 
            bit_ask.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            # Hedge trasnaction in orgin exchange (Kraken)
            
            krak_bid['accum_size'] = krak_bid['bid_size'].cumsum()
            krak_bid['remaining_vol'] = vol_to_fill - krak_bid['accum_size']
            to_drop = krak_bid[krak_bid['remaining_vol']>0]
            
            krak_bid.drop(to_drop.index, inplace=True)
            krak_bid.reset_index(drop=True, inplace=True)
            
            new_vol = -krak_bid.iloc[0,-1] # remaing volume on surviving level
            
            krak_bid.iloc[0,0] = new_vol
            krak_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            krak_bid.reset_index(drop=True, inplace=True)
            
        else:
            
            bit_bid.rename(columns={'bid_size':'size','bid':'price'}, inplace=True)
            
            bit_bid = bit_bid.merge(pd.DataFrame(to_fill[['size','price']]).transpose(), 
                                    on=['size','price'], how='outer'
                                   ).sort_values(by='price', ascending=False, ignore_index=True)   
            
            bit_bid.rename(columns={'size':'bid_size','price':'bid'}, inplace=True)

            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
    else:
        
        # identify order's fee structure
        fee_type = 'taker' if to_fill['price'] < bit_bid.iloc[0]['bid'] else 'maker'            
        
        if fee_type == 'taker':
            
            bit_bid['accum_size'] = bit_bid['bid_size'].cumsum()


            vol_to_fill = to_fill['size']

            bit_bid['remaining_vol'] = vol_to_fill - bit_bid['accum_size']
            to_drop = bit_bid[bit_bid['remaining_vol']>0]

            bit_bid.drop(to_drop.index, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)

            new_vol = -bit_bid.iloc[0,-1] # remaing volume on surviving level

            bit_bid.iloc[0,0] = new_vol
            bit_bid.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            bit_bid.reset_index(drop=True, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)
            
            # Hedge transaction in origin market (Kraken)
            krak_ask['accum_size'] = krak_ask['bid_size'].cumsum()
            krak_ask['remaining_vol'] = vol_to_fill - krak_ask['accum_size']
            to_drop = krak_ask[krak_ask['remaining_vol']>0]
            
            krak_ask.drop(to_drop.index, inplace=True)
            krak_ask.reset_index(drop=True, inplace=True)
            
            new_vol = -krak_ask.iloc[0,-1] # remaing volume on surviving level
            
            krak_ask.iloc[0,1] = new_vol
            krak_ask.drop(['accum_size','remaining_vol'], axis=1, inplace=True)
            krak_ask.reset_index(drop=True, inplace=True)
            
            
        else:
            bit_ask.rename(columns={'ask_size':'size','ask':'price'}, inplace=True)
            
            bit_ask = bit_ask.merge(pd.DataFrame(to_fill[['price','size']]).transpose(), 
                                    on=['price','size'], how='outer'
                                   ).sort_values(by='price', ascending=True, ignore_index=True) 
            
            bit_ask.rename(columns={'size':'ask_size','price':'ask'}, inplace=True)
            
            levels_added.drop(0, inplace=True)
            levels_added.reset_index(drop=True, inplace=True)

C:\Users\if724\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-364-cb687d1d62e9>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  krak_bid['accum_size'] = krak_bid['bid_size'].cumsum()


In [380]:
bit_ob = pd.concat([bit_bid, bit_ask], axis=1)

In [412]:
krak_ask.reset_index(drop=True, inplace=True)
krak_ob = pd.concat([krak_bid, krak_ask], axis=1)

# Next steps

1. Insertar liquidez en OB:

    * Proceso de matching engine con ask subsecuente (órdenes límite que crucen el spread hasta que sean agotadas) (taker fee)
    * Calcular comisiones cada vez que se empaten órdenes
    * Calcular efectos de nuestras órdenes y crear un nuevo Orderbook
    
 
2. Comparar OB creado vs OB en siguiente timestamp  

    * Obtener diferencias en niveles para validar si se llenaron nuestras órdenes con el siguiente OB (maker fee)


3. Incorporar proceso de hedge en origin exchange

    * Cada vez que se ejecute una orden en destination, se hae una operación contraria a spot en el origin
    
    
4. Reflejar efectos de hedge en el origin exchange

    * Preguntar al profe